# University Advising Ontology KBS


## Ontology Design

This Knowledge-Based System (KBS) models a simplified university advising scenario.

### Concepts (Classes)
- **Student**
- **Course**

### Relationships (Properties)
- **requires(Course → Course)**  
  Represents prerequisite relationships between courses.

- **completed(Student → Course)**  
  Indicates courses successfully completed by a student.

- **eligibleFor(Student → Course)** *(Inferred)*  
  A student is eligible if all prerequisites are completed.

### Inference Goal
The system determines:

1. Whether a student can take a course
2. Which courses should be recommended next


In [1]:
class UniversityOntologyKBS:
    def __init__(self):
        self.students = set()
        self.courses = set()
        self.prerequisites = {}   # course -> set(prereqs)
        self.completed = {}       # student -> set(courses)

    # -----------------------------
    # Entity Management
    # -----------------------------

    # Add students, courses, and prerequisites
    def add_student(self, student: str):
        self.students.add(student)
        self.completed.setdefault(student, set())

    def add_course(self, course: str):
        self.courses.add(course)
        self.prerequisites.setdefault(course, set())

    def add_prerequisite(self, course: str, prereq: str):
        if course not in self.courses or prereq not in self.courses:
            raise ValueError("Unknown course")
        self.prerequisites[course].add(prereq)

# Mark a course as completed for a student
    def complete_course(self, student: str, course: str):
        if student not in self.students:
            raise ValueError("Unknown student")
        if course not in self.courses:
            raise ValueError("Unknown course")
        self.completed[student].add(course)

    # -----------------------------
    # Inference Logic
    # -----------------------------


    # Helper method to get all prerequisites for a course (including indirect)
    def _get_all_prereqs(self, course: str, visited=None):
        if visited is None:
            visited = set()

        for prereq in self.prerequisites.get(course, set()):
            if prereq not in visited:
                visited.add(prereq)
                self._get_all_prereqs(prereq, visited)

        return visited

# Check if a student can take a course based on completed prerequisites
    def can_take(self, student: str, course: str):
        if student not in self.students:
            raise ValueError("Unknown student")
        if course not in self.courses:
            raise ValueError("Unknown course")

        all_prereqs = self._get_all_prereqs(course)
        completed_courses = self.completed[student]

        missing = all_prereqs - completed_courses
        eligible = len(missing) == 0

        return eligible, missing
# Recommend courses that a student is eligible to take based on completed courses
    def recommend_courses(self, student: str):
        if student not in self.students:
            raise ValueError("Unknown student")

        recommendations = []

        for course in self.courses:
            if course in self.completed[student]:
                continue

            eligible, _ = self.can_take(student, course)
            if eligible:
                recommendations.append(course)

        return sorted(recommendations)


## Dataset

Define students, courses, and prerequisite relationships.
The dataset satisfies:

- ≥ 6 courses, 8 unique courses.
- ≥ 3 students, 5 students.
- ≥ 7 prerequisite links
- Multi-level prerequisites


In [2]:
kbs = UniversityOntologyKBS()

# ---- Courses ----
courses = [
    "General Anatomy",
    "Human Physiology",
    "General Pathology",
    "Pharmacology",
    "Clinical Medicine",
    "Medical Imaging",
    "Oncology",
    "Surgery"
]

# Add courses to the KBS
for c in courses:
    kbs.add_course(c)

# ---- Prerequisites ----
kbs.add_prerequisite("Human Physiology", "General Anatomy")

kbs.add_prerequisite("General Pathology", "Human Physiology")

kbs.add_prerequisite("Pharmacology", "Human Physiology")

kbs.add_prerequisite("Clinical Medicine", "General Pathology")

kbs.add_prerequisite("Medical Imaging", "General Anatomy")

kbs.add_prerequisite("Surgery", "Clinical Medicine")

kbs.add_prerequisite("Oncology", "General Pathology")
kbs.add_prerequisite("Oncology", "Pharmacology")

# ---- Students ----
# Add students to the KBS
students = ["Nashepae", "Ngaruiya", "Munge", "Kago", "Mugure"]
for s in students:
    kbs.add_student(s)

# ---- Completed Courses ----
# Nashepae → int-progress
kbs.complete_course("Nashepae", "General Anatomy")
kbs.complete_course("Nashepae", "Human Physiology")

# Mugure → mid-progress
kbs.complete_course("Mugure", "General Anatomy")
kbs.complete_course("Mugure", "Human Physiology")
kbs.complete_course("Mugure", "General Pathology")


# Ngaruiya → advanced
kbs.complete_course("Ngaruiya", "General Anatomy")
kbs.complete_course("Ngaruiya", "Human Physiology")
kbs.complete_course("Ngaruiya", "General Pathology")
kbs.complete_course("Ngaruiya", "Pharmacology")

# Munge → most advanced
kbs.complete_course("Munge", "General Anatomy")
kbs.complete_course("Munge", "Human Physiology") 
kbs.complete_course("Munge", "General Pathology")
kbs.complete_course("Munge", "Pharmacology")
kbs.complete_course("Munge", "Clinical Medicine")

# Kago → beginner (none completed)


## Inference Demo

Evaluate student eligibility using inferred prerequisite chains.


In [3]:

checks = [
    ("Nashepae", "General Pathology"),     # Should be eligible
    ("Nashepae", "Oncology"),              # Missing Pathology + Pharmacology
    ("Mugure", "Clinical Medicine"),       # Should be eligible
    ("Mugure", "Oncology"),                # Missing Pharmacology
    ("Kago", "Human Physiology")           # Missing Anatomy
]

# Check eligibility for each student-course pair

for student, course in checks:
    eligible, missing = kbs.can_take(student, course)

    print(f"{student} → {course}")
    print(f"Eligible: {eligible}")

    if not eligible:
        print(f"Missing prerequisites: {missing}")

    print("-" * 40)


Nashepae → General Pathology
Eligible: True
----------------------------------------
Nashepae → Oncology
Eligible: False
Missing prerequisites: {'General Pathology', 'Pharmacology'}
----------------------------------------
Mugure → Clinical Medicine
Eligible: True
----------------------------------------
Mugure → Oncology
Eligible: False
Missing prerequisites: {'Pharmacology'}
----------------------------------------
Kago → Human Physiology
Eligible: False
Missing prerequisites: {'General Anatomy'}
----------------------------------------


## Recommendations

Courses are recommended if:

- The student has NOT completed them
- The student is eligible based on prerequisites


In [4]:

print("\nCourse Recommendations:")

# Recommend courses for each student
for student in students:
    recs = kbs.recommend_courses(student)
    print(f"{student}: {recs}")
    


Course Recommendations:
Nashepae: ['General Pathology', 'Medical Imaging', 'Pharmacology']
Ngaruiya: ['Clinical Medicine', 'Medical Imaging', 'Oncology']
Munge: ['Medical Imaging', 'Oncology', 'Surgery']
Kago: ['General Anatomy']
Mugure: ['Clinical Medicine', 'Medical Imaging', 'Pharmacology']


## Unit Tests

Validate ontology behavior and inference correctness.


In [5]:
# Unit tests for the KBS
import unittest

class TestKBS(unittest.TestCase):

# Set up a fresh KBS instance for each test
    def setUp(self):
        self.kbs = UniversityOntologyKBS()
        for c in ["A", "B", "C"]:
            self.kbs.add_course(c)

        self.kbs.add_prerequisite("B", "A")
        self.kbs.add_prerequisite("C", "B")

        self.kbs.add_student("Mararo")

# Test that students and courses are added correctly

    def test_student_added(self):
        self.assertIn("Mararo", self.kbs.students)

    def test_course_added(self):
        self.assertIn("A", self.kbs.courses)

    def test_prereq_inference(self):
        prereqs = self.kbs._get_all_prereqs("C")
        self.assertEqual(prereqs, {"A", "B"})

    def test_not_eligible(self):
        eligible, missing = self.kbs.can_take("Mararo", "C")
        self.assertFalse(eligible)
        self.assertEqual(missing, {"A", "B"})

    def test_becomes_eligible(self):
        self.kbs.complete_course("Mararo", "A")
        self.kbs.complete_course("Mararo", "B")
        eligible, _ = self.kbs.can_take("Mararo", "C")
        self.assertTrue(eligible)

    def test_recommendation(self):
        self.kbs.complete_course("Mararo", "A")
        recs = self.kbs.recommend_courses("Mararo")
        self.assertIn("B", recs)
        
# Run the unit tests
unittest.TextTestRunner().run(
    unittest.defaultTestLoader.loadTestsFromTestCase(TestKBS)
)


......
----------------------------------------------------------------------
Ran 6 tests in 0.023s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>